# Procesamiento del lenguaje natural - 2025 - B4 - Desafio 1
**Inteligencia Artificial - CEIA - FIUBA**

## Autor

- **Mendoza Dante**.
- **SIU: e2206**.

**Nota:** tome como base el codigo compartido por los docentes.

In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [ ]:
# ========================================================================================
# Metodos de ayuda para mostrar documentos y vecinos
# ========================================================================================

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

X_train_raw = newsgroups_train.data

def topk_similares_de_documento(doc_index: int, X, k: int = 5):
    """
    Devuelve los índices y similitudes de los k documentos más similares a doc_index dentro de X (no incluye el propio).
    """
    sims = cosine_similarity(X[doc_index], X).ravel()
    sims[doc_index] = -1.0  # excluir el propio documento
    top_idx = np.argpartition(sims, -k)[-k:]
    top_idx = top_idx[np.argsort(-sims[top_idx])]
    return [(int(i), float(sims[i])) for i in top_idx]

def mostrar_documento(idx, data, label, max_chars=400):
    """
    Para mostrar un documento con su índice, clase y un fragmento del texto.
    """
    print(f"[idx={idx} | clase={label}]")
    texto = data[idx].replace("\n", " ")
    print("Texto:", texto[:max_chars] + ("..." if len(texto) > max_chars else ""))
    print("-"*80)

# ========================================================================================
# Selección aleatoria de 5 documentos y visualización de vecinos
# ========================================================================================

import random
num_muestras = min(5, X_train.shape[0])
muestras = random.sample(range(X_train.shape[0]), num_muestras)

for di in muestras:
    print("\n" + "="*100)
    print(f"Documento base:")
    mostrar_documento(di, X_train_raw, y_train[di])

    vecinos = topk_similares_de_documento(di, X_train, k=5)
    print("Vecinos más similares:")
    for j, s in vecinos:
        print(f"Similitud={s:.3f}")
        mostrar_documento(j, X_train_raw, y_train[j])



Documento base:
[idx=8127 | clase=5]
Texto:   You wanna do masking. Build a bitmap (pixmap of depth one) where all pixels you name "opaque" are 1 (that get copied) and the others are 0. Use this bitmap as the clip_mask in the gc used for XCopyArea(), and remember to adjust the clip_origin coordinates to the XCopyArea() blit origin.  The Mouse pointer (besides from that it is driven using RAMDAC analog mapping on most hardwares) uses a mask,...
--------------------------------------------------------------------------------
Vecinos más similares:
Similitud=0.356
[idx=4133 | clase=5]
Texto:  No. As soon as you blit two of this icons once on top of the other with a little dislocation, you see the rectangular blit crashes too much of the icon first blitted, because it draws a full rectangle. The way to do it is masking: Create a bitmap with all pixels to be merged are 1 and all not to be merged are 0. Then, set the clip_mask of the gc to this bitmap, set the clip_x_origin and clip_y_or...

### Análisis de resultados

Se seleccionaron 5 documentos al azar del conjunto de entrenamiento y se calcularon sus 5 vecinos más similares utilizando **TF-IDF + similitud coseno**.

- En varios casos, los documentos más similares pertenecen a la **misma clase** y comparten vocabulario temático (ejemplo: hardware, deportes).  
- En otros, aparecen documentos de **clases distintas**, especialmente cuando el texto contiene lenguaje más **genérico o ambiguo** (ejemplo: debates políticos, religiosos o filosóficos).  
- Las similitudes numéricas suelen ser **moderadas-bajas**, lo que refleja que TF-IDF captura bien la coincidencia de palabras clave pero **no el significado semántico**.

**Conclusión:**  
TF-IDF funciona de manera aceptable cuando el tema está bien definido por su vocabulario, pero muestra limitaciones al enfrentar textos con lenguaje común o temas superpuestos.


**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [ ]:
# ========================================================================================
# 2. Clasificación por prototipos (vecino más similar en TRAIN)
# ========================================================================================

from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Normalizo filas para usar producto punto como similitud coseno
Xtr_norm = normalize(X_train, norm='l2', copy=True)
Xte_norm = normalize(X_test, norm='l2', copy=True)

# Similaridad coseno entre todos los test y todos los train
S = Xte_norm @ Xtr_norm.T

# Para cada test, buscamos el train con mayor similitud
y_pred_proto_idx = np.asarray(S.argmax(axis=1)).ravel()
y_pred_proto = np.array([y_train[j] for j in y_pred_proto_idx], dtype=int)

f1_macro_proto = f1_score(y_test, y_pred_proto, average='macro')
print("\n=== Clasificación por prototipos (1-NN por similitud coseno) ===")
print(f"F1-macro (TEST): {f1_macro_proto:.4f}")

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_proto, zero_division=0))

print("Matriz de confusión (primeras 10 filas/columnas):")
cm = confusion_matrix(y_test, y_pred_proto)
lim = min(cm.shape[0], 10)
print(cm[:lim, :lim])



=== Clasificación por prototipos (1-NN por similitud coseno) ===
F1-macro (TEST): 0.5050

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.37      0.51      0.43       319
           1       0.54      0.48      0.51       389
           2       0.51      0.46      0.48       394
           3       0.52      0.52      0.52       392
           4       0.53      0.50      0.52       385
           5       0.70      0.59      0.64       395
           6       0.63      0.46      0.53       390
           7       0.41      0.58      0.48       396
           8       0.63      0.52      0.57       398
           9       0.65      0.54      0.59       397
          10       0.75      0.72      0.73       399
          11       0.55      0.59      0.57       396
          12       0.53      0.33      0.41       393
          13       0.65      0.49      0.56       396
          14       0.64      0.51      0.57       394
          15      

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


In [ ]:
# ========================================================================================
# 3. Naive Bayes (MultinomialNB y ComplementNB)
# ========================================================================================

from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold

cv_scheme = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

modelos = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

# Grilla de hiperparámetros
param_grid = {"alpha": [0.1, 0.5, 1.0]}

resultados_nb = {}

print("\n=== Naive Bayes sobre TF-IDF ya vectorizado ===")
for nombre, clf in modelos.items():
    gs = GridSearchCV(
        estimator=clf,
        param_grid={"alpha": [0.1, 0.5, 1.0]},
        scoring="f1_macro",
        cv=cv_scheme,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    best = gs.best_estimator_
    print(f"\nModelo: {nombre}")
    print(f"Mejores params: {gs.best_params_}")
    print(f"Mejor F1-macro (CV): {gs.best_score_:.4f}")
    # Evaluación en TEST
    y_pred = best.predict(X_test)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    print(f"F1-macro (TEST): {f1_macro:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))
    resultados_nb[nombre] = {
        "best_params": gs.best_params_,
        "cv_f1_macro": gs.best_score_,
        "test_f1_macro": f1_macro
    }



=== Naive Bayes sobre TF-IDF ya vectorizado ===
Fitting 3 folds for each of 3 candidates, totalling 9 fits

Modelo: MultinomialNB
Mejores params: {'alpha': 0.1}
Mejor F1-macro (CV): 0.7054
F1-macro (TEST): 0.6565
              precision    recall  f1-score   support

           0       0.69      0.23      0.35       319
           1       0.68      0.69      0.69       389
           2       0.70      0.52      0.60       394
           3       0.61      0.75      0.67       392
           4       0.77      0.69      0.73       385
           5       0.83      0.75      0.79       395
           6       0.85      0.73      0.78       390
           7       0.81      0.75      0.78       396
           8       0.82      0.73      0.77       398
           9       0.94      0.81      0.87       397
          10       0.60      0.93      0.73       399
          11       0.61      0.79      0.69       396
          12       0.72      0.57      0.64       393
          13       0.85      

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [ ]:
# ========================================================================================
# 4. Vectorización de palabras (término-documento) + vecinos semánticos
# ========================================================================================

# Uso la matriz TF-IDF de ENTRENAMIENTO ya calculada.
from sklearn.preprocessing import normalize

TDM = X_train.T.tocsr()
terms = tfidfvect.get_feature_names_out().tolist()

print(f"\nMatriz término-documento: {TDM.shape} (términos x docs)")

# Palabras semilla
PALABRAS_SEMILLA_MANUAL = ["extirpation", "costas", "wholsale", "cedex", "toivanen"]

print("\n=== Palabras semilla utilizadas ===")
print(PALABRAS_SEMILLA_MANUAL)

# Vecinos similares
TDM_norm = normalize(TDM, norm='l2', copy=True)

def vecinos_termino(termino: str, TDM_norm, terms, topk: int = 5):
    if termino not in terms:
        return []
    idx = terms.index(termino)
    v = TDM_norm[idx]
    sims = (TDM_norm @ v.T).toarray().ravel()
    sims[idx] = -1.0
    top_idx = np.argpartition(sims, -topk)[-topk:]
    top_idx = top_idx[np.argsort(-sims[top_idx])]
    return [(terms[i], float(sims[i])) for i in top_idx]

print("\n=== Vecinos más similares por palabra (top-5) ===")
for w in PALABRAS_SEMILLA_MANUAL:
    nn = vecinos_termino(w, TDM_norm, terms, topk=5)
    print(f"\nPalabra: {w}")
    for rank, (tw, s) in enumerate(nn, start=1):
        print(f"  {rank:>2}. {tw:<20} sim={s:.4f}")



Matriz término-documento: (101631, 11314) (términos x docs)

=== Palabras semilla utilizadas ===
['extirpation', 'costas', 'wholsale', 'cedex', 'toivanen']

=== Vecinos más similares por palabra (top-5) ===

Palabra: extirpation
   1. salzburg             sim=1.0000
   2. judenevakuierung     sim=1.0000
   3. nuremberg            sim=1.0000
   4. ausrottung           sim=1.0000
   5. rje                  sim=1.0000

Palabra: costas
   1. assesment            sim=1.0000
   2. interjects           sim=1.0000
   3. fatso                sim=1.0000
   4. knicks               sim=1.0000
   5. blanketing           sim=1.0000

Palabra: wholsale
   1. deliberatly          sim=1.0000
   2. nunber               sim=1.0000
   3. authorise            sim=1.0000
   4. overordering         sim=1.0000
   5. resellers            sim=0.9936

Palabra: cedex
   1. stol                 sim=0.9745
   2. titusville           sim=0.9745
   3. stennis              sim=0.9745
   4. 9075                 sim=0.9

### Análisis de resultados

Se utilizó la **matriz término-documento** (TF-IDF transpuesta) para estudiar la similitud semántica entre palabras.  
Se eligieron manualmente 5 palabras semilla:

- `extirpation`  
- `costas`  
- `wholsale`  
- `cedex`  
- `toivanen`  

Para cada palabra se calcularon sus **5 vecinos más similares** usando **similitud coseno** sobre las filas normalizadas de la matriz término-documento.

#### Observaciones generales

- Las palabras devuelven vecinos que aparecen en contextos o documentos similares, lo que refleja relaciones semánticas.
- Algunos términos poco frecuentes muestran similitudes perfectas (1.0) con palabras raras que aparecen en los mismos documentos.
